# Training a simple linear probe on a transformer model

In this introductory notebook, we will train a simple linear probe for a transformer model to check if causal language modelling understanding for the wikitext dataset is present in a transformer model even at some intermediate layer.

In [1]:
# Standard imports
from transformers import (
    AutoTokenizer,
    MistralForCausalLM,
    Trainer,
    BitsAndBytesConfig,
    TrainingArguments,
    GPT2Model,
    GPT2LMHeadModel,
)
from datasets import load_dataset
from peft import LoraConfig
import torch

# Imports from the transformer_heads library
from transformer_heads import load_headed
from transformer_heads.util.helpers import DataCollatorWithPadding, get_model_params
from transformer_heads.config import HeadConfig
from transformer_heads.util.model import print_trainable_parameters
from transformer_heads.util.evaluate import evaluate_head_wise, get_top_n_preds

Set the model and it's model parameters. Default is GPT2, but you can also use Mistral 7b if you have enough GPU RAM (and are willing to wait longer for training to complete)

In [2]:
# Parameters
model_path = "meta-llama/Llama-2-7b-hf"
train_epochs = 1
eval_epochs = 1
logging_steps = 100

In [3]:
model_params = get_model_params(model_path)
model_class = model_params["model_class"]
hidden_size = model_params["hidden_size"]
vocab_size = model_params["vocab_size"]
print(model_params)

{'vocab_size': 32000, 'max_position_embeddings': 4096, 'hidden_size': 4096, 'intermediate_size': 11008, 'num_hidden_layers': 32, 'num_attention_heads': 32, 'num_key_value_heads': 32, 'hidden_act': 'silu', 'initializer_range': 0.02, 'rms_norm_eps': 1e-05, 'pretraining_tp': 1, 'use_cache': True, 'rope_theta': 10000.0, 'rope_scaling': None, 'attention_bias': False, 'attention_dropout': 0.0, 'mlp_bias': False, 'return_dict': True, 'output_hidden_states': False, 'output_attentions': False, 'torchscript': False, 'torch_dtype': 'float16', 'use_bfloat16': False, 'tf_legacy_loss': False, 'pruned_heads': {}, 'tie_word_embeddings': False, 'chunk_size_feed_forward': 0, 'is_encoder_decoder': False, 'is_decoder': False, 'cross_attention_hidden_size': None, 'add_cross_attention': False, 'tie_encoder_decoder': False, 'max_length': 20, 'min_length': 0, 'do_sample': False, 'early_stopping': False, 'num_beams': 1, 'num_beam_groups': 1, 'diversity_penalty': 0.0, 'temperature': 1.0, 'top_k': 50, 'top_p': 1

Start out by configuring the linear probing head. In this example we hook at layer -4. This is using the python indexing format. E.g. gpt-2 has 12 transformer blocks. Hooking at layer -4 means that the linear probe processes the hidden state after the 9th layer while hooking at layer -1 would mean processing the hidden state after the last (12th) transformer block.

Otherwise, we are setting *num_layers* to 1, to make sure that we are actually training a *linear* probe and not an mlp. With *is_causal_lm* we specify the type of task that the model is supposed to learn.

In [4]:
heads_configs = [
    HeadConfig(
        name="wikitext_head",
        layer_hook=-4,  # Hook to layer [-4] (Drop 3 layers from the end)
        in_size=hidden_size,
        num_layers=1,
        output_activation="linear",
        is_causal_lm=True,
        loss_fct="cross_entropy",
        num_outputs=vocab_size,
        is_regression=False,
        output_bias=False,
    )
]

Now we load and format our dataset. We need to make sure that the dataset has labels stored for each head that we want to train. In case of the causal language modelling task, these labels are just copys of the input_ids.

In [5]:
dd = load_dataset("wikitext", "wikitext-2-v1")

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token


def tokenize_function(examples):
    out = tokenizer(examples["text"], padding=False, truncation=True)
    out[heads_configs[0].name] = out["input_ids"].copy()
    return out


for split in dd.keys():
    dd[split] = dd[split].filter(function=lambda example: len(example["text"]) > 10)
    dd[split] = dd[split].map(tokenize_function, batched=True)
dd.set_format(
    type="torch", columns=["input_ids", "attention_mask", heads_configs[0].name]
)
for split in dd.keys():
    dd[split] = dd[split].remove_columns("text")

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Filter:   0%|          | 0/4358 [00:00<?, ? examples/s]

...

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Filter:   0%|          | 0/36718 [00:00<?, ? examples/s]

Map:   0%|          | 0/23627 [00:00<?, ? examples/s]

Filter:   0%|          | 0/3760 [00:00<?, ? examples/s]

Map:   0%|          | 0/2460 [00:00<?, ? examples/s]

In [7]:
dd["train"]

Dataset({
    features: ['input_ids', 'attention_mask', 'wikitext_head'],
    num_rows: 23627
})

Now it is time to load our model. The load_headed function of transformer_heads is great for loading frozen models with a linear probe. To save GPU memory, we will load the model in a quantized state. 

In [8]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    load_in_8bit=False,
    llm_int8_threshold=6.0,
    llm_int8_has_fp16_weight=False,
    bnb_4bit_compute_dtype=torch.float32,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

model = load_headed(
    model_class,
    model_path,
    head_configs=heads_configs,
    quantization_config=quantization_config,
    device_map={"": torch.cuda.current_device()},
)

TypeError: unsupported operand type(s) for |: 'type' and 'ABCMeta'

That warning about weights not initialized from the model checkpoint is exactly what we want to see. We want a newly initialized linear probe that is not in the pretrained gpt2 checkpoint.

Let's check some data about our model using the convenience method *print_trainable_parameters*.

In [10]:
print_trainable_parameters(model)

all params: 3500412928 || trainable params: 131072000 || trainable%: 3.7444725149866662
params by dtype: defaultdict(<class 'int'>, {torch.float32: 262410240, torch.uint8: 3238002688})
trainable params by dtype: defaultdict(<class 'int'>, {torch.float32: 131072000})


The linear probe has vocab_size * hidden_size (32000*4096) parameters. So already quite a few, but nothing compared to 7 billion other parameters of the model.

Let's see how our linear probe performs before it is trained

In [11]:
print(
    get_top_n_preds(
        n=5, model=model, text="The historical significance of", tokenizer=tokenizer
    )
)

{'wikitext_head': ['imi', 'directories', 'units', 'Harold', 'buffer']}


As expected, this is pretty random.

Let's train the linear probe now using huggingfaces simple to use Trainer class. Note that we are using a custom collator here, to handle the labels under the heads_configs names correctly.

In [12]:
args = TrainingArguments(
    output_dir="linear_probe_test",
    learning_rate=0.0002,
    num_train_epochs=train_epochs,
    logging_steps=logging_steps,
    do_eval=False,
    remove_unused_columns=False,  # Important to set to False, otherwise things will fail
)
collator = DataCollatorWithPadding(
    feature_name_to_padding_value={
        "input_ids": tokenizer.pad_token_id,
        heads_configs[0].name: -100,
        "attention_mask": 0,
    }
)
trainer = Trainer(
    model,
    args=args,
    train_dataset=dd["train"],
    data_collator=collator,
)
trainer.train()

wandb: Currently logged in as: ykeller (chm-hci). Use `wandb login --relogin` to force relogin


wandb: wandb version 0.16.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


wandb: Tracking run with wandb version 0.16.3


wandb: Run data is saved locally in /raven/u/ykeller/transformer_heads/wandb/run-20240324_052940-6dqq3bhi
wandb: Run `wandb offline` to turn off syncing.


wandb: Syncing run resilient-water-216


...

Checkpoint destination directory linear_probe_test/checkpoint-2000 already exists and is non-empty.Saving will proceed but saved results may be invalid.


/u/ykeller/conda-envs/sh_finetuning/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Checkpoint destination directory linear_probe_test/checkpoint-2500 already exists and is non-empty.Saving will proceed but saved results may be invalid.


/u/ykeller/conda-envs/sh_finetuning/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


TrainOutput(global_step=2954, training_loss=3.3226569349409036, metrics={'train_runtime': 17031.5515, 'train_samples_per_second': 1.387, 'train_steps_per_second': 0.173, 'total_flos': 2.7756254601621504e+17, 'train_loss': 3.3226569349409036, 'epoch': 1.0})

So this is nice to see, the probe is learning something and the training loss decreases. But how about evaluation on the validation set?

In [13]:
print(evaluate_head_wise(model, dd["validation"], collator, epochs=eval_epochs))


Evaluating: 100%|██████████| 308/308 [09:36<00:00,  1.87s/it]

/u/ykeller/conda-envs/sh_finetuning/lib/python3.10/site-packages/torch/utils/checkpoint.py:90: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


(2.8574375010930098, {'wikitext_head': 2.8574375010930098})


Yep, the evaluation loss is similar to the training loss, indicating no overfitting. How do things look in a practical example?

In [14]:
print(get_top_n_preds(5, model, "The historical significance of", tokenizer))

{'wikitext_head': ['the', '', 'Old', 'these', 'St']}


We see that the linear probe has learned to predict tokens that are pretty likely to follow that sentence.